[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-vector-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-vector-generation.ipynb)

# SPLADE Sparse-Dense Embedding Generation

## Overview

SPLADE is a class of models that produce sparse embeddings. Dense embeddings are often difficult to interpret, but sparse embeddings have clearly identifiable token overlap, making sparse vector search results more interpretable. SPLADE models have been shown to consistently outperform dense models, particularly in out-of-domain settings. 

The following guide will show you how to construct SPLADE embeddings to use in Pinecone's sparse-dense vectors. See the [companion guide to skip embedding generation](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).

## Prerequisites

We'll install the required libraries:

In [1]:
!pip install -qU \
          pinecone-text \
          sentence_transformers \
          tqdm \
          pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.6 MB/s eta 0:00:00


In [12]:
from tqdm.notebook import tqdm

## Quora Dataset

We'll load the popular Quora dataset:

In [2]:
import pandas as pd

df = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/quora_all-MiniLM-L6-v2_Splade/raw/quora_questions_sample200.parquet")

In [3]:
df.head()

,id,text
0,17248,"If I fall under the Brady law due to PTSD, is..."
1,240419,Which question can't be answered with a yes o...
2,262372,How can I write a children's book for older k...
3,180057,What happens when you view a public Instagram...
4,456610,What is the fact about NIBIRU the Planet X?


### Sparse Embeddings with SPLADE 

In the following example we will use the [naver/splade-cocondenser-ensembledistil](https://huggingface.co/naver/splade-cocondenser-ensembledistil) SPLADE model.


In [6]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on {device}")

running on cpu


In [8]:
from pinecone_text.sparse import Splade

splade = Splade(device=device)

In [9]:
doc = "what is the capital of france?"
sparse_vector = splade(doc)


### Dense Model

We use the popular all-MiniLM-L6-v2 model available on Hugging Face for dense vectors.

In [10]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

### Compute Dense & Sparse Embeddings

Create BM25 sparse embeddings:

In [14]:
sparse_values = []

batch_size = 20
for i in tqdm(range(0, len(df), batch_size)):
  sparse_values += splade(df.iloc[i:i + batch_size]["text"].tolist())

df['sparse_values'] = sparse_values

  0%|          | 0/10 [00:00<?, ?it/s]

And now encode dense vector embeddings:

In [15]:
batch_size = 20
dense_values = []
for i in tqdm(range(0, len(df), batch_size)):
  dense_values += model.encode(df.iloc[i:i + batch_size]["text"].tolist()).tolist()

df['values'] = dense_values

  0%|          | 0/10 [00:00<?, ?it/s]

We organize our dataframe to align to the `pinecone-datasets` format:

In [16]:
df_result = df.copy()
df_result["metadata"] = None
df_result["blob"] = df_result["text"].apply(lambda t: {"text": t})
df_result = df_result.drop(columns="text")

In [17]:
df_result.head()

,id,sparse_values,values,metadata,blob
0,17248,"{'indices': [2012, 2017, 2046, 2065, 2076, 207...","[0.021123137325048447, 0.04391808062791824, -0...",None,{'text': ' If I fall under the Brady law due t...
1,240419,"{'indices': [1005, 1012, 1029, 1037, 1998, 200...","[0.015179820358753204, 0.06904049962759018, -0...",None,{'text': ' Which question can't be answered wi...
2,262372,"{'indices': [1005, 1012, 1029, 1037, 1055, 200...","[0.038049325346946716, 0.08449704945087433, 0....",None,{'text': ' How can I write a children's book f...
3,180057,"{'indices': [1012, 2006, 2008, 2017, 2037, 204...","[-0.028053391724824905, -0.04008294269442558, ...",None,{'text': ' What happens when you view a public...
4,456610,"{'indices': [1012, 1029, 1060, 1996, 2055, 205...","[0.0056223189458251, 0.10487580299377441, 0.02...",None,{'text': ' What is the fact about NIBIRU the P...


And now we have all we need to start using Pinecone vector database 🚀

For more details on that, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).